Imports

In [ ]:
import utils.scrape_no_js as bourso
import utils.inputs as inputs
import utils.helper_functions as hf

from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=np.inf)

Config

In [ ]:
buy_time = 'Open'
loss_limit = 0.997

cumulated_probs_target = 0.85

fee = 0.002

download_days = 500 #should be > 151 = 90 (input days) * 7/5 (days per week) * 1.2 (margin in case of vacations)

model_path = './outputs/classifier_model.keras'
scaler_path = './outputs/scaler.pkl'

In [ ]:
tickers_file_path = './db/tickers_euronext_regulated_euro_500k€.xlsx'
df_tickers = pd.read_excel(tickers_file_path)

num_tickers = len(df_tickers)
print(f'number of tickers: {num_tickers}')

Download past data

In [ ]:
def get_past_date(days_past):
    today = date.today()
    past_date = today - timedelta(days=days_past)
    formatted_date = past_date.strftime('%Y-%m-%d')
    return formatted_date

def days_difference(date1, date2): # can handle date objects and 'YYYY-MM-DD' strings
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)

    difference = abs((date2 - date1).days)
    return difference

In [ ]:
start_date = get_past_date(download_days)
today = get_past_date(0)

print(f'start download date: {start_date}')
print(f'today: {today}')

In [ ]:
yahoo_tickers = df_tickers['yahoo'].iloc[1:num_tickers+1].values.tolist()

download_past_data = yf.download(yahoo_tickers, start=start_date, end=today)
df_download_past = pd.DataFrame(download_past_data)
df_download_past_filled = hf.fillnavalues(df_download_past)
# df_download = df_download.dropna(axis='columns', how='all') #remove columns with only nan values

print(df_download_past_filled.tail(3).to_markdown())


In [ ]:
last_index = df_download_past_filled.index[-1]
last_download_past_date = last_index.date()
print(f'last download date: {last_download_past_date}')

download_missing_days = days_difference(today, last_download_past_date)
print(f'download missing days: {download_missing_days}')

# if download_missing_days > 1:
#     raise ValueError("Too many download missing days")


Download today's data

In [ ]:
bourso_tickers = df_tickers['bourso'].iloc[1:num_tickers+1].values.tolist()

df_download_today = bourso.get_theoretical_opening_prices(bourso_tickers)

print(df_download_today)

Transform past data

In [ ]:
def get_single_level_df(df, ohlcv):
    new_df = df[[ohlcv]]
    new_df = hf.remove_top_column_name(new_df)

    return new_df

def get_ohlcv_dfs(df):
    df_open = get_single_level_df(df, 'Open')
    df_high = get_single_level_df(df, 'High')
    df_low = get_single_level_df(df, 'Low')
    df_close = get_single_level_df(df, 'Close')
    df_volume = get_single_level_df(df, 'Volume')
    
    return {'df_open': df_open, 'df_high': df_high, 'df_low': df_low,
            'df_close': df_close, 'df_volume': df_volume}

In [ ]:
list_dfs_ohlcv = get_ohlcv_dfs(df_download_past_filled)
df_transformed_past = get_single_level_df(df_download_past_filled, buy_time)

print(df_transformed_past.tail(3).to_markdown())
print(df_transformed_past.shape)

Transform today's data

In [ ]:
mapping_dict = dict(zip(df_tickers['bourso'], df_tickers['yahoo']))

df_download_today['ticker'] = df_download_today['ticker'].map(mapping_dict)

print(df_download_today.tail(3))


In [ ]:
df_transformed_today = df_download_today.set_index('ticker').T

today_date = datetime.now().strftime("%Y-%m-%d 00:00:00")
df_transformed_today.index = [today_date]

df_transformed_today = df_transformed_today.reindex(sorted(df_transformed_today.columns), axis='columns')
df_transformed_today = df_transformed_today.rename_axis('Date')

print(df_transformed_today.to_markdown())
print(df_transformed_today.shape)

Transform: Assemble past and today's data

In [ ]:
df_transformed = pd.concat([df_transformed_past, df_transformed_today], axis=0)

print(df_transformed.tail(3).to_markdown())

Transform: Fill missing values

In [ ]:
df_transformed_filled = hf.fillnavalues(df_transformed)

print(df_transformed_filled.tail(3).to_markdown())
print(df_transformed_filled.shape)

Transform: Fill open, high, low, close, volume today's missing values (nan values)

In [ ]:
for key in ['df_high', 'df_low', 'df_close', 'df_volume']:
    list_dfs_ohlcv[key].loc[today_date] = np.nan

print(list_dfs_ohlcv['df_high'].tail(3).to_markdown())
print(list_dfs_ohlcv['df_high'].shape)

Transform: Create inputs

In [ ]:
df_inputs = inputs.get_inputs(df_transformed_filled, list_dfs_ohlcv)

last_day_index = df_inputs.index.get_level_values(0)[-1] # Get the last value in the top-level index
df_inputs_last_day = df_inputs.loc[last_day_index]

X_inputs_last_day = df_inputs_last_day.values # Convert DataFrame to numpy array

print(df_inputs_last_day.tail(3).to_markdown())

In [ ]:
print(f'shape of X_inputs_last_day (rows -> tested data, columns -> inputs): {X_inputs_last_day.shape}')

In [ ]:
scaler = hf.load_object(scaler_path)

X_inputs_last_day_scaled = scaler.transform(X_inputs_last_day)

Load model

In [ ]:
model = tf.keras.models.load_model(model_path)

Predict

In [ ]:
prediction = model.predict(X_inputs_last_day_scaled)
df_prediction = pd.DataFrame(prediction, columns=['buy', 'do_not_buy'], index=df_inputs_last_day.index)
# df_prediction = df_prediction['buy']
df_prediction = df_prediction.sort_values(by='do_not_buy')


print(df_prediction.to_markdown())


Evaluate

In [ ]:
# df_open = list_dfs_ohlcv['df_open']
# df_close = list_dfs_ohlcv['df_close']

# df_open_last_day = df_open.iloc[-1]
# df_close_last_day = df_close.iloc[-1]

# df_real = pd.DataFrame()
# df_real['variation'] = df_close_last_day / df_open_last_day

In [ ]:
# df_low = list_dfs_ohlcv['df_low']
# df_low_last_day = df_low.loc[last_day_index]
# df_real['lowest_variation'] = df_low_last_day / df_open_last_day

# df_real['profit'] = df_real['variation']
# df_real.loc[df_real['lowest_variation'] < loss_limit, 'profit'] = loss_limit
# df_real['profit'] *= hf.get_fee_coef(fee)

In [ ]:
# df_evaluation = pd.concat([df_prediction, df_real], axis='columns')
# print(df_evaluation)

In [ ]:
# df_filtered_evaluation = df_evaluation[df_evaluation['buy'] > cumulated_probs_target]

# count = df_filtered_evaluation['profit'].count()
# average = df_filtered_evaluation['profit'].mean()
# median = df_filtered_evaluation['profit'].median()

# print(f'count: {count}')
# print(f'average profit: {average}')
# print(f'median profit: {median}')

In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(12, 7))
# plt.scatter(x=df_filtered_evaluation['buy'], y=df_filtered_evaluation['variation'], color='blue', alpha=0.3, s=5)
# plt.title('variation vs buy prediction')
# plt.xlabel('buy prediction')
# plt.ylabel('real variation')
# plt.grid(True)
# plt.show()